Read dữ liệu

In [1]:
# !unzip "/content/drive/MyDrive/5_AIADVANCE/data.zip" -d "/content/drive/MyDrive/5_AIADVANCE"

Archive:  /content/drive/MyDrive/5_AIADVANCE/data.zip
   creating: /content/drive/MyDrive/5_AIADVANCE/data/
  inflating: /content/drive/MyDrive/5_AIADVANCE/data/data_test.txt  
  inflating: /content/drive/MyDrive/5_AIADVANCE/data/data_train.txt  
  inflating: /content/drive/MyDrive/5_AIADVANCE/data/y_test_v1.txt  
  inflating: /content/drive/MyDrive/5_AIADVANCE/data/y_train_v1.txt  


In [1]:
import os
os.chdir("/content/drive/MyDrive/5_AIADVANCE/")

In [2]:
import numpy as np
import pandas as pd
import urllib.request
from numpy import linalg as LA
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.utils import to_categorical

In [3]:
def loaddata(path,fr):
  print("/content/gdrive/My Drive/data_mean/"+path)
  df = pd.read_csv(path, sep="\t", index_col=0)
  dfs = df.iloc[:, fr:].apply(lambda x: x, axis=0)
  return dfs


# /content/gdrive/My Drive/c3d_24_2
def init(_trainX, _trainY, _trainL, _testX, _testY, _testL):
  return (_trainX, _trainY, _trainL, _testX, _testY, _testL)

In [4]:
_trainX = loaddata("data/data_train.txt",1)
_trainY = loaddata("data/y_train_v1.txt",0)
_trainL = len(_trainY.values)
_testX = loaddata("data/data_test.txt",1)
_testY = loaddata("data/y_test_v1.txt",0)
_testL = len(_testY.values)
print(_trainL)
print(_testL)

/content/gdrive/My Drive/data_mean/data/data_train.txt
/content/gdrive/My Drive/data_mean/data/y_train_v1.txt
/content/gdrive/My Drive/data_mean/data/data_test.txt
/content/gdrive/My Drive/data_mean/data/y_test_v1.txt
11979
7892


In [5]:
_trainX

,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,...,col114,col115,col116,col117,col118,col119,col120,col121,col122,col123
0,-9.245019,40.771074,457.598681,119.034278,19.044164,379.562914,-519.012475,41.857416,-744.442426,86.806006,...,253.003710,-50.978784,-609.406645,208.191943,175.071631,39.503664,171.889819,-61.570580,51.181879,307.864062
1,-9.278103,40.881477,457.408763,119.363254,18.973510,379.554027,-517.506619,42.782577,-745.628148,87.587742,...,253.538036,-50.559231,-609.748879,208.454661,175.219578,39.161430,171.790477,-61.060574,51.292282,307.855296
2,-8.849720,40.994075,457.254188,119.610608,18.905056,379.490028,-515.448841,43.800472,-747.140633,87.834973,...,254.017127,-50.130847,-609.998349,208.843299,175.195202,38.730861,171.726478,-60.270130,51.314357,307.610145
3,-8.874029,40.994079,457.329259,119.586298,18.724002,379.565099,-513.571905,44.796287,-748.966670,87.720209,...,254.363805,-49.974127,-610.179436,209.099400,175.442621,38.459274,171.801549,-59.389044,51.495422,307.413609
4,-9.481243,40.680541,457.572241,119.431721,18.591521,379.445899,-511.825481,46.202796,-750.534357,87.565632,...,254.516089,-49.766521,-610.402406,209.342261,175.469075,38.326796,171.863379,-59.453167,51.543997,307.747046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11974,-49.214233,-54.168295,638.830877,61.215332,23.041849,558.905218,-218.750610,-52.539022,-785.438052,-52.291748,...,-53.073298,74.249634,-302.453634,-68.460993,74.068604,56.894816,377.873357,-99.993652,-73.267172,416.704656
11975,-50.867768,-54.949829,638.607919,60.104766,21.898254,558.682259,-229.908417,-60.652344,-782.945548,-52.316010,...,-54.020377,69.428008,-304.592896,-68.231376,72.867583,55.751221,377.469247,-101.828217,-73.958191,416.572152
11976,-52.355793,-55.678388,638.089117,59.340984,20.717120,557.982427,-240.900593,-68.531659,-780.386820,-52.174763,...,-55.263300,65.133953,-306.588666,-68.569147,72.194256,54.751116,376.950445,-102.501667,-75.229841,416.324957
11977,-52.772973,-56.786664,637.605578,58.561622,19.699359,557.408434,-253.084863,-76.157026,-777.521223,-53.587670,...,-56.651991,61.639136,-308.602094,-69.143201,71.143409,53.823870,376.467028,-102.194726,-76.338117,415.660387


In [6]:
# Đây là thuật toán PCA
# Input: Dữ liệu đầu vào là một ma trận X (n x m) với n là số lượng mẫu, m là số chiều của mỗi mẫu
# Output: Ma trận X sau khi giảm chiều về k chiều (n x k)
# Cách sử dụng:
# pca = PCA(X, k)
# pca.fit()
# X_new = pca.transform(X)
# Trong đó X_new là ma trận X sau khi giảm chiều về k chiều
class PCA:
    def __init__(self, X, k):
        self.X = X
        self.k = k
        self.n, self.m = X.shape
        self.mean = np.mean(X, axis=0)
        self.X = self.X - self.mean
        self.cov = np.dot(self.X.T, self.X) / self.n
        self.eigenvalues, self.eigenvectors = np.linalg.eig(self.cov)
        self.eigenvalues_idx = np.argsort(self.eigenvalues)[::-1]

    def fit(self):
        self.eigenvalues = np.real(self.eigenvalues)
        self.eigenvectors = np.real(self.eigenvectors)
        self.eigenvectors = self.eigenvectors[:, self.eigenvalues_idx]
        self.eigenvectors = self.eigenvectors[:, :self.k]

    def transform(self, X):
        return np.dot(X - self.mean, self.eigenvectors)

    def inverse_transform(self, X_new):
        return np.dot(X_new, self.eigenvectors.T) + self.mean

    # tính phương sai
    def variance(self):
        return np.sum(self.eigenvalues[:self.k]) / np.sum(self.eigenvalues)

    def get_eigenvectors(self):
        return self.eigenvectors

    def get_mean(self):
        return self.mean

In [ ]:
# ######### function of pca #################
# def PCA(dfs, fn):
#   X = dfs.values
#   mu = Sum(X)/len(X)
#   f=[]
#   for x in X:
#     f.append(x-mu)
#   x = np.transpose(f);
#   A = np.dot(x,f)
#   w, v = LA.eig(A)
#   idx = w.argsort()[::-1]
#   w = w[idx]
#   v = v[:,idx]
#   if fn != 0:
#     v = np.delete(v, np.s_[-fn:], axis=1)
#   B = np.dot(f,v)
#   #B = pd.DataFrame(B, columns=["PC{}".format(x + 1) for x in range(len(dfs.columns))])
#   return B,w,v,mu

In [ ]:
# def conv(dfs,mu,v):
#   X = dfs.values
#   f=[]
#   for x in X:
#     f.append(x-mu)
#   B = np.dot(f,v)
#   #B = pd.DataFrame(B, columns=["PC{}".format(x + 1) for x in range(len(dfs.columns))])
#   return B
# ######### end of function of pca ##########
# def run_pca(_trainX, _trainY, _trainL, _testX, _testY, _testL, dim = 1): # dim = 11
#   trainX, trainY, trainL, testX, testY, testL = init(_trainX, _trainY, _trainL, _testX, _testY, _testL)
#   trainX,w,v,mu = PCA(trainX, dim) # pre_PCA
#   #trainX = pre_PCA(trainX)
#   X = array(trainX).reshape(trainL, 1, len(trainX[0]))
#   Y = [x - 1 for x in trainY.values]
#   Y = to_categorical(Y)[:trainL]


## Model

In [13]:
print(len(X))
print(X[0])
print(len(Y))
print(Y[0])

11979
[[  -9.24501894   40.77107414  457.59868101  119.03427794   19.0441645
   379.56291441 -519.01247499   41.85741599 -744.44242617   86.80600645
    88.47973808  600.45305113  -97.51052187   67.83917411   43.61058286
   254.46530821  393.56170829  496.25444273  158.14279845   -9.11028687
   632.13823179  537.63938048  349.11209281  580.89885191  369.16538145
   395.28177055  539.70830015   55.84519103  191.32043632  -69.64094302
  -512.76601503  -17.25787656 -713.48159548 -207.95559022  140.53378471
  -446.87434145 -548.34389589   52.53981193 -896.07813784 -393.08693788
   109.93506034 -576.23997256   55.84519103  163.89021476  448.27421812
  -161.0617914   186.06976493 -334.98078677 -490.67702577   13.61243614
  -915.90393969  149.8140631   317.51754936  454.06804136  434.07443907
   265.28244193  550.66227964  438.60092833  405.51150688  598.3708978
     5.42063048   50.45765098 -888.11160036  -69.62770937  -37.08367936
   365.34990171  -27.98439882   29.18338569  589.31804136  -

In [7]:
trainX, trainY, trainL, testX, testY, testL = init(_trainX, _trainY, _trainL, _testX, _testY, _testL)
# trainX,w,v,mu = PCA(trainX, dim) # pre_PCA
#trainX = pre_PCA(trainX)
X = array(trainX).reshape(trainL, 1, len(trainX.iloc[0]))
Y = [x - 1 for x in trainY.values]
Y = to_categorical(Y)[:trainL]

In [15]:
model = Sequential()
model.add(LSTM(1024, activation='tanh', input_shape=(1, len(trainX.iloc[0]))))
model.add(Dense(120, activation='tanh'))
model.add(Dense(len(Y[0]), activation='tanh'))
model.add(Activation("softmax"))
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 1024)              4702208   
                                                                 
 dense_2 (Dense)             (None, 120)               123000    
                                                                 
 dense_3 (Dense)             (None, 8)                 968       
                                                                 
 activation_1 (Activation)   (None, 8)                 0         
                                                                 
Total params: 4826176 (18.41 MB)
Trainable params: 4826176 (18.41 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [19]:
model.fit(X, Y, epochs=30, batch_size=32)

Epoch 1/30
375/375 [==============================] - 3s 7ms/step - loss: 0.8524 - accuracy: 0.8667
Epoch 2/30
375/375 [==============================] - 2s 6ms/step - loss: 0.8538 - accuracy: 0.8648
Epoch 3/30
375/375 [==============================] - 3s 7ms/step - loss: 0.8464 - accuracy: 0.8708
Epoch 4/30
375/375 [==============================] - 3s 9ms/step - loss: 0.8585 - accuracy: 0.8553
Epoch 5/30
375/375 [==============================] - 4s 10ms/step - loss: 0.8698 - accuracy: 0.8608
Epoch 6/30
375/375 [==============================] - 4s 10ms/step - loss: 0.8659 - accuracy: 0.8522
Epoch 7/30
375/375 [==============================] - 2s 6ms/step - loss: 0.8649 - accuracy: 0.8576
Epoch 8/30
375/375 [==============================] - 2s 5ms/step - loss: 0.8379 - accuracy: 0.8774
Epoch 9/30
375/375 [==============================] - 2s 6ms/step - loss: 0.8330 - accuracy: 0.8720
Epoch 10/30
375/375 [==============================] - 2s 6ms/step - loss: 0.8568 - accuracy: 0.86

In [21]:
# testX = conv(testX,mu,v)
testX = array(testX).reshape(len(testX), 1, len(testX.iloc[0]))
testY = [x-1 for x in testY.values]
testY = to_categorical(testY)[:testL]
score = model.evaluate(testX, testY, verbose=1)
print("evaluate loss: {0[0]}\nevaluate acc: {0[1]}".format(score))

247/247 [==============================] - 1s 4ms/step - loss: 1.0697 - accuracy: 0.7091
evaluate loss: 1.0697044134140015
evaluate acc: 0.709072470664978


In [ ]:
# DIM = _trainX.shape[1]
# i = 1
# while i < DIM:
#   print('PCA i: ', i)
#   run_pca(_trainX, _trainY, _trainL, _testX, _testY, _testL, i)
#   print('end i: ', i)
#   i += 1